## *Types of networks, SIR epidemic on different networks (Part 1)*

The aim of this tutorial is to investigate what the influence of network structure is on the spread of an epidemic. We will look at various types of networks and an agent based SIR model running on these networks. We will compare network structure in terms of degree distribution and centrality measures. Epidemic outcome will be measured as final size, duration, and peak prevalence of the outbreak.

First we need to set up a directory structure to store data, output, and figures.

In [26]:
#Navigate to the directory containing this file
    cd(@__DIR__)

# Create directories if they don't exist
    if !isdir("data")
        mkdir("data")
    end
    if !isdir("figures")
        mkdir("figures")
    end
    if !isdir("output")
        mkdir("output")
    end

Then we activate the current directory as the environment for Pkg. We import the packages we will need to run the model. This might take some time.

In [27]:
 # Import the Pkg module, activate the current directory as the environment for Pkg, instantiate the environment
    using Pkg
    Pkg.activate(@__DIR__)
    Pkg.instantiate()

  # Import the necessary packages
    using Agents, Graphs, Random, Plots, DataFrames, CSV, Statistics, StatsBase, Distributions, Measures

  Activating project at `c:\Users\leost\julia-workspace\espidam-tutorial-2025`


A degree distribution to be used with the configuration model is read from a csv file. The file needs to have a number of rows that equals the population size of the network. For every node an integer degree is given. 

In [28]:
 
    # Read in deg_dist.csv to use for the random_configuration_model
    degrees = CSV.read("deg_dist.csv", DataFrame, header=false)

    # Display the first 10 entries of the degree distribution
    println("First 10 entries of the degree distribution:")
    first(degrees, 10)

First 10 entries of the degree distribution:


Row,Column1
,Int64
1,6
2,16
3,9
4,5
5,3
6,5
7,3
8,5
9,3


Next, the model is defined. The code is provided in various source files that are included. In the source file create_graph.jl a network is set up with a given number of nodes (default size 1000) and a given network structure. Then the structure of the agents is defined. An agent is characterized by its infection status (susceptible, infected, or recovered), by its infection age (time since infection), and by risk category (low or high). The source file initialize.jl describes the initialization procedure of the model, and the agent_step.jl describes events that are performed per time step. Then, in model_step! the numbers of individuals in disease states are summed up.

In [29]:

include("src/create_graph.jl")

# Agent creation: agents of type Person and properties status, days_infected and risk
@agent struct Person(GraphAgent)
    status::Symbol = :S #((S)usceptible, (I)nfected, (R)ecovered)
    days_infected::Int = 0 # number of days since infection
    risk::Symbol = :high # something to differentiate agents (here, high and low risk)
end

include("src/initialize.jl")
include("src/agent_step.jl")

# Model step: keep track of the infection numbers
function model_step!(model::ABM)
    model.susceptible_count = sum([model[i].status == :S for i in 1:nv(model.graph)])
    model.infected_count = sum([model[i].status == :I for i in 1:nv(model.graph)])
    model.recovered_count = sum([model[i].status == :R for i in 1:nv(model.graph)])
end

model_step! (generic function with 1 method)

Now let's start with setting up the model for our investigation. In the initialization function, we can choose what network type we want to use, where in the network the infection is seeded, what proportion of the population is high risk, and what the mean degree of the network is.

In [30]:
# Initialization: initialize the model with the chosen parameters: network_type, patient_zero, high_risk, mean_degree
model = initialize(; network_type=:random, patient_zero=:random, mean_degree=2)

StandardABM with 1000 agents of type Person
 agents container: Dict
 space: GraphSpace with 1000 positions and 988 edges
 scheduler: fastest
 properties: days_to_recovered, trans_prob, mean_degree, high_risk, susceptible_count, fraction_high_risk, infected_count, recovered_count, graph, dispersion, n_nodes, patient_zero, network_type

## Network Analysis

The constructed network can be analyzed using our `analyze_graph` function, which returns a comprehensive set of network metrics. The function calculates:

- **Basic metrics**: density, mean degree, clustering coefficient, assortativity
- **Component analysis**: number and sizes of connected components
- **Centrality measures**: degree, betweenness, closeness, and eigenvector centrality
- **Structural properties**: degree distribution, maximal cliques, diameter

The results are organized in a dictionary with easily accessible components for further analysis and visualization.

In [32]:
include("src/analyze_graph.jl")

# Analyze the graph
graph_analysis = analyze_graph(model.graph)

# Display the analysis results
analysis_display = print_graph_analysis(graph_analysis)

# Display each component of the analysis
display("Graph Summary Metrics")
display(analysis_display.summary)

display("Centrality Measures Statistics")
display(analysis_display.centrality)

display("Component Size Summary")
display(analysis_display.component_summary)

# Save the centrality measures to a CSV file for further analysis
CSV.write("data/centrality_$(model.network_type)_mdeg_$(model.mean_degree).csv", graph_analysis["centrality"])

# Save summary metrics to a CSV file
CSV.write("data/graph_summary_$(model.network_type)_mdeg_$(model.mean_degree).csv", graph_analysis["summary"])

"Graph Summary Metrics"

Row,metric,value
,String,Any
1,Mean Degree,1.976
2,Density,0.00197798
3,Clustering Coefficient,0.00152749
4,Assortativity,0.000460549
5,Diameter,Graph is not connected
6,Number of Connected Components,174
7,Max Component Length,793


"Centrality Measures Statistics"

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Float64,Float64,Float64,Int64,DataType
1,degree_centrality,0.00197798,0.0,0.002002,0.00700701,0,Float64
2,betweenness_centrality,0.00503555,0.0,0.00166072,0.0419485,0,Float64
3,closeness_centrality,0.0713634,0.0,0.0862548,0.118694,0,Float64
4,eigenvector_centrality,0.0171674,1.48575e-13,0.00717348,0.212361,0,Float64


"Component Size Summary"

Row,component_size,count
,Int64,Int64
1,1,148
2,2,20
3,3,3
4,4,1
5,6,1
6,793,1


"data/graph_summary_random_mdeg_2.csv"

### Network Visualization and Analysis

We can use the results from our network analysis to create visualizations that help us understand the structure of the network. Let's look at the degree distribution and explore how centrality measures relate to epidemic dynamics.

In [ ]:
include("src/plotting.jl")

# Plot the degree distribution using our stored degree distribution data
plotdegdist = plot_degree_distribution(graph_analysis["degree_distribution"]; network_type=model.network_type)
display(plotdegdist)
savefig(plotdegdist, "figures/plotdegdist_$(model.network_type)_mdeg_$(model.mean_degree).pdf")

# Plot centrality distribution
p1 = histogram(graph_analysis["centrality"].degree_centrality, title="Degree Centrality", legend=false)
p2 = histogram(graph_analysis["centrality"].betweenness_centrality, title="Betweenness Centrality", legend=false)
p3 = histogram(graph_analysis["centrality"].closeness_centrality, title="Closeness Centrality", legend=false)
p4 = histogram(graph_analysis["centrality"].eigenvector_centrality, title="Eigenvector Centrality", legend=false)

centrality_plot = plot(p1, p2, p3, p4, layout=(2,2), size=(800, 600))
display(centrality_plot)

In [ ]:
# specify "adata" (agent data to collect) and "mdata" (model data to collect)
adata = [:status]
mdata = [:susceptible_count, :infected_count, :recovered_count]

adf, mdf = run!(model, 100; adata, mdata)

In [ ]:
plotdynamics = plot_epidemic_trajectories(mdf)
display(plotdynamics)
savefig(plotdynamics, "figures/plotdynamics_$(model.network_type)_mdeg_$(model.mean_degree)_nn_$(model.n_nodes)_disp_$(model.dispersion)_pat0_$(model.patient_zero)_hirisk_$(model.high_risk)_hr_frac_$(model.fraction_high_risk)_trans_$(model.trans_prob).pdf")


## Single Run Visualization

Let's run a single simulation and inspect both the epidemic dynamics plot and the degree distribution plot. The `plot_single_run` function returns both plots and a combined plot, making it easy to examine them together.

In [ ]:
# Get both plots from plot_single_run
dynamics_plot, degdist_plot, combined_plot = plot_single_run(; network_type=:random, mean_degree=2)

display(combined_plot)

In [ ]:
include("src/run_simulations.jl")
# Run the simulation multiple times with different seeds (100 times), output results to file and plot
result_plot = run_and_plot(; plot_type=:sfr, network_type=:preferentialattachment, patient_zero=:maxdegree, mean_degree=2)
display(result_plot)